In [1]:
import sys
import os

# Get current working directory
current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

#Insert the path to the parent directory
sys.path.insert(0, parent_dir)

c:\Users\Yayerad\Telecom\notebooks
c:\Users\Yayerad\Telecom


In [2]:
# Import the custom loader script
from scripts.db_loader import load_data

In [3]:
# Load data from the database into a DataFrame
df = load_data()

# Check if data is loaded
if df is not None:
    print("Data loaded successfully.")
    print(df.head())  # Display the first few rows of the data

Data loaded successfully.
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          2024739

 Calculate Engagement and Experience Scores

In [4]:
import pandas as pd
from scripts.satisfaction import (
    calculate_engagement_experience_scores,
    build_regression_model,
    evaluate_regression_model,
    kmeans_clustering
)
from sklearn.model_selection import train_test_split

In [5]:
import numpy as np
#  Calculate Engagement and Experience Scores
engagement_centroids = np.array([[1000, 1000, 1000000, 500000]])
experience_centroids = np.array([[50, 50, 100, 100]])

df['engagement_score'], df['experience_score'] = calculate_engagement_experience_scores(df, engagement_centroids, experience_centroids)

# 3. Calculate satisfaction score
df['satisfaction_score'] = (df['engagement_score'] + df['experience_score']) / 2

In [7]:
#  Calculate satisfaction score
df['satisfaction_score'] = (df['engagement_score'] + df['experience_score']) / 2

In [9]:
#  Split data for regression model
X = df[['Activity Duration DL (ms)', 'Activity Duration UL (ms)', 'Total DL (Bytes)', 'Total UL (Bytes)',
        'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']]
y = df['satisfaction_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
#  Build regression model
model = build_regression_model(X_train, y_train)

In [11]:
# Evaluate regression model
rmse = evaluate_regression_model(model, X_test, y_test)
print(f"Regression Model RMSE: {rmse}")

Regression Model RMSE: 1519492.3145041154


In [12]:
#  Perform K-means clustering on engagement and experience scores
df = kmeans_clustering(df, n_clusters=2)

In [13]:

#  Aggregate satisfaction and experience scores per cluster
cluster_agg = df.groupby('cluster').agg({
    'satisfaction_score': 'mean',
    'experience_score': 'mean'
}).reset_index()

print("Cluster Aggregation Results:")
print(cluster_agg)




Cluster Aggregation Results:
   cluster  satisfaction_score  experience_score
0        0        3.332052e+08      13554.742993
1        1        1.238214e+08      13649.741193
